<a href="https://colab.research.google.com/github/Hanbin-git/dacon_new_drug/blob/main/20250710(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip -o "/content/drive/MyDrive/data.zip" -d "/content/data_1"

Archive:  /content/drive/MyDrive/data.zip
  inflating: /content/data_1/sample_submission.csv  
  inflating: /content/data_1/test.csv  
  inflating: /content/data_1/train.csv  


In [ ]:
# ==============================================================================
# ✅ 초기 환경 설정 및 라이브러리 설치 (가장 먼저 실행)
# ==============================================================================

# 1. pip 캐시를 모두 비웁니다.
print("Clearing pip cache...")
!pip cache purge

# 2. numpy와 scikit-learn 관련 패키지를 강제로 제거합니다.
print("Uninstalling numpy and scikit-learn related packages...")
!pip freeze | grep -E 'numpy|scikit-learn' | xargs -n 1 pip uninstall -y

# 3. Colab 기본 라이브러리 (TensorFlow, Numba 등)와 호환되는 특정 버전의 numpy와 scikit-learn을 설치합니다.
#    numpy를 1.26.x 대로 낮춰서 시도합니다.
print("Installing compatible versions of numpy and scikit-learn (attempting numpy 1.26.x)...")
!pip install numpy==1.26.4 scikit-learn==1.6.0 --no-cache-dir --quiet

# 4. 나머지 필요한 라이브러리들을 설치합니다.
print("Installing other necessary libraries...")
!pip install rdkit-pypi catboost xgboost lightgbm optuna --no-cache-dir --quiet

# 5. 파이썬 런타임을 강제로 재시작합니다. (가장 중요!)
print("Forcing runtime restart... Please wait a few seconds for reconnection.")
import os
os.kill(os.getpid(), 9)

# ==============================================================================

Clearing pip cache...
Files removed: 0
Uninstalling numpy and scikit-learn related packages...
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
Installing compatible versions of numpy and scikit-learn (attempting numpy 1.26.x)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 141.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 330.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 175.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.

In [ ]:
# import pandas as pd
# import os

# # ✅ Colab 경로
# extract_path = "/content/data_1"
# def get_path(filename):
#     return os.path.join(extract_path, filename)

# # ✅ 데이터 불러오기
# train = pd.read_csv(get_path("train.csv"))
# test = pd.read_csv(get_path("test.csv"))

# # ✅ 컬럼 확인
# print("✅ [train.csv] 컬럼 목록:")
# print(train.columns.tolist())

# print("\n✅ [test.csv] 컬럼 목록:")
# print(test.columns.tolist())

# # ✅ 샘플 데이터 미리 보기
# print("\n✅ [train.csv] 데이터 샘플:")
# print(train.head())

# print("\n✅ [test.csv] 데이터 샘플:")
# print(test.head())


In [ ]:
# # ✅ import
# import pandas as pd
# import numpy as np
# import os
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Crippen, Lipinski, rdMolDescriptors, AllChem
# from sklearn.preprocessing import StandardScaler
# from catboost import CatBoostRegressor, Pool
# from lightgbm import LGBMRegressor
# from xgboost import XGBRegressor
# from sklearn.linear_model import Ridge
# from sklearn.model_selection import KFold

# # ✅ 경로 설정 (Colab 환경)
# extract_path = "/content/data_1"
# def get_path(filename):
#     return os.path.join(extract_path, filename)

# # ✅ 데이터 로딩
# train = pd.read_csv(get_path("train.csv"))
# test = pd.read_csv(get_path("test.csv"))
# submission = pd.read_csv(get_path("sample_submission.csv"))

# # ✅ RDKit Feature 추출
# def extract_rdkit_features(df):
#     mols = [Chem.MolFromSmiles(smi) for smi in df['Canonical_Smiles']]
#     features = {
#         'MolWt': [Descriptors.MolWt(mol) if mol else np.nan for mol in mols],
#         'LogP': [Crippen.MolLogP(mol) if mol else np.nan for mol in mols],
#         'NumHDonors': [Lipinski.NumHDonors(mol) if mol else np.nan for mol in mols],
#         'NumHAcceptors': [Lipinski.NumHAcceptors(mol) if mol else np.nan for mol in mols],
#         'TPSA': [rdMolDescriptors.CalcTPSA(mol) if mol else np.nan for mol in mols],
#         'NumRotatableBonds': [Descriptors.NumRotatableBonds(mol) if mol else np.nan for mol in mols],
#         'RingCount': [mol.GetRingInfo().NumRings() if mol else np.nan for mol in mols],
#         'HeavyAtomCount': [mol.GetNumHeavyAtoms() if mol else np.nan for mol in mols],
#         'FractionCSP3': [rdMolDescriptors.CalcFractionCSP3(mol) if mol else np.nan for mol in mols],
#         'NumAliphaticRings': [rdMolDescriptors.CalcNumAliphaticRings(mol) if mol else np.nan for mol in mols],
#         'NumAromaticRings': [rdMolDescriptors.CalcNumAromaticRings(mol) if mol else np.nan for mol in mols]
#     }
#     return pd.DataFrame(features)

# # ✅ Morgan Fingerprint 추출 (2048-bit)
# def get_morgan_fingerprint(smiles, radius=2, nBits=2048):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol:
#         fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
#         return np.array(fp)
#     else:
#         return np.zeros(nBits)

# def extract_morgan_df(df, nBits=2048):
#     fps = df['Canonical_Smiles'].apply(lambda x: get_morgan_fingerprint(x, nBits=nBits))
#     return pd.DataFrame(fps.tolist(), columns=[f'MFP_{i}' for i in range(nBits)])

# # ✅ Feature 생성
# X_train_rdkit = extract_rdkit_features(train)
# X_test_rdkit = extract_rdkit_features(test)
# X_train_morgan = extract_morgan_df(train)
# X_test_morgan = extract_morgan_df(test)

# # ✅ 결합
# X_train = pd.concat([X_train_rdkit, X_train_morgan], axis=1)
# X_test = pd.concat([X_test_rdkit, X_test_morgan], axis=1)
# y_train = train["Inhibition"]

# # ✅ 결측치 처리
# X_train.fillna(0, inplace=True)
# X_test.fillna(0, inplace=True)

# # ✅ 정규화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
# X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# # ✅ Sample Weight
# sample_weight = np.log1p(y_train)

# # ✅ Stacking 앙상블
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# oof_cat = np.zeros(len(X_train))
# oof_lgb = np.zeros(len(X_train))
# oof_xgb = np.zeros(len(X_train))
# test_cat = np.zeros(len(X_test))
# test_lgb = np.zeros(len(X_test))
# test_xgb = np.zeros(len(X_test))

# for train_idx, val_idx in kf.split(X_train_df):
#     X_tr, X_val = X_train_df.iloc[train_idx], X_train_df.iloc[val_idx]
#     y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
#     w_tr = sample_weight.iloc[train_idx]

#     # CatBoost
#     cat = CatBoostRegressor(iterations=1500, learning_rate=0.03, depth=6,
#                             verbose=0, early_stopping_rounds=100, random_state=42)
#     cat.fit(Pool(X_tr, y_tr, weight=w_tr), eval_set=Pool(X_val, y_val))
#     oof_cat[val_idx] = cat.predict(X_val)
#     test_cat += cat.predict(X_test_df) / kf.n_splits

#     # LightGBM
#     lgb = LGBMRegressor(n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42)
#     lgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_lgb[val_idx] = lgb.predict(X_val)
#     test_lgb += lgb.predict(X_test_df) / kf.n_splits

#     # XGBoost
#     xgb = XGBRegressor(n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42)
#     xgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_xgb[val_idx] = xgb.predict(X_val)
#     test_xgb += xgb.predict(X_test_df) / kf.n_splits

# # ✅ 메타 모델
# stacked_train = np.vstack([oof_cat, oof_lgb, oof_xgb]).T
# stacked_test = np.vstack([test_cat, test_lgb, test_xgb]).T
# meta_model = Ridge(alpha=1.0)
# meta_model.fit(stacked_train, y_train)
# final_preds = meta_model.predict(stacked_test)

# # ✅ 제출 파일 저장
# submission['Inhibition'] = final_preds
# submission.to_csv('submission_stacking_rdkit_morgan.csv', index=False)
# print("✅ 'submission_stacking_rdkit_morgan.csv' 생성 완료!")


In [ ]:
# import pandas as pd
# import numpy as np
# import os
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Crippen, Lipinski, rdMolDescriptors, AllChem
# from sklearn.preprocessing import StandardScaler
# from catboost import CatBoostRegressor, Pool

# # ✅ 데이터 로딩
# extract_path = "/content/data_1"  # Colab 기준
# train = pd.read_csv(os.path.join(extract_path, "train.csv"))
# test = pd.read_csv(os.path.join(extract_path, "test.csv"))
# submission = pd.read_csv(os.path.join(extract_path, "sample_submission.csv"))

# # ✅ RDKit 기반 화학 특성 추출 (13개 내외)
# def extract_physchem_features(df):
#     mols = [Chem.MolFromSmiles(smi) for smi in df['Canonical_Smiles']]
#     features = {
#         'MolWt': [Descriptors.MolWt(mol) if mol else np.nan for mol in mols],
#         'LogP': [Crippen.MolLogP(mol) if mol else np.nan for mol in mols],
#         'NumHDonors': [Lipinski.NumHDonors(mol) if mol else np.nan for mol in mols],
#         'NumHAcceptors': [Lipinski.NumHAcceptors(mol) if mol else np.nan for mol in mols],
#         'TPSA': [rdMolDescriptors.CalcTPSA(mol) if mol else np.nan for mol in mols],
#         'NumRotatableBonds': [Descriptors.NumRotatableBonds(mol) if mol else np.nan for mol in mols],
#         'RingCount': [mol.GetRingInfo().NumRings() if mol else np.nan for mol in mols],
#         'HeavyAtomCount': [mol.GetNumHeavyAtoms() if mol else np.nan for mol in mols],
#         'FractionCSP3': [rdMolDescriptors.CalcFractionCSP3(mol) if mol else np.nan for mol in mols],
#         'NumAliphaticRings': [rdMolDescriptors.CalcNumAliphaticRings(mol) if mol else np.nan for mol in mols],
#         'NumAromaticRings': [rdMolDescriptors.CalcNumAromaticRings(mol) if mol else np.nan for mol in mols],
#     }
#     return pd.DataFrame(features)

# # ✅ Morgan Fingerprint (2048 bit)
# def get_morgan_fp(smiles, radius=2, nBits=2048):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol:
#         return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits))
#     else:
#         return np.zeros(nBits)

# def extract_morgan_df(df, nBits=2048):
#     fps = df['Canonical_Smiles'].apply(lambda x: get_morgan_fp(x, nBits=nBits))
#     return pd.DataFrame(fps.tolist(), columns=[f'MFP_{i}' for i in range(nBits)])

# # ✅ 피처 생성
# X_train_phys = extract_physchem_features(train)
# X_test_phys = extract_physchem_features(test)
# X_train_fp = extract_morgan_df(train)
# X_test_fp = extract_morgan_df(test)

# # ✅ 피처 결합
# X_train = pd.concat([X_train_phys, X_train_fp], axis=1)
# X_test = pd.concat([X_test_phys, X_test_fp], axis=1)
# y_train = train["Inhibition"]

# # ✅ 정규화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # ✅ Sample Weighting (log scale)
# sample_weight = np.log1p(y_train)

# # ✅ CatBoost 학습
# model = CatBoostRegressor(
#     iterations=1500,
#     learning_rate=0.03,
#     depth=6,
#     loss_function='RMSE',
#     early_stopping_rounds=100,
#     verbose=100,
#     random_state=42
# )

# model.fit(Pool(X_train_scaled, y_train, weight=sample_weight))
# preds = model.predict(X_test_scaled)

# # ✅ 제출 파일 저장
# submission["Inhibition"] = preds
# submission.to_csv("submission_catboost_morgan_physchem13.csv", index=False)
# print("✅ 'submission_catboost_morgan_physchem13.csv' 저장 완료")


In [ ]:
# # ✅ 라이브러리
# import os
# import numpy as np
# import pandas as pd
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Crippen, Lipinski, rdMolDescriptors, AllChem
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import Ridge
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# from catboost import CatBoostRegressor, Pool
# from lightgbm import LGBMRegressor
# from xgboost import XGBRegressor

# # ✅ 경로 설정 (Colab 기준)
# extract_path = "/content/data_1"
# def get_path(filename): return os.path.join(extract_path, filename)

# train = pd.read_csv(get_path("train.csv"))
# test = pd.read_csv(get_path("test.csv"))
# submission = pd.read_csv(get_path("sample_submission.csv"))

# # ✅ RDKit 기반 분자 특성 추출
# def extract_rdkit_features(df):
#     mols = [Chem.MolFromSmiles(smi) for smi in df['Canonical_Smiles']]
#     features = {
#         'MolWt': [Descriptors.MolWt(mol) if mol else np.nan for mol in mols],
#         'LogP': [Crippen.MolLogP(mol) if mol else np.nan for mol in mols],
#         'NumHDonors': [Lipinski.NumHDonors(mol) if mol else np.nan for mol in mols],
#         'NumHAcceptors': [Lipinski.NumHAcceptors(mol) if mol else np.nan for mol in mols],
#         'TPSA': [rdMolDescriptors.CalcTPSA(mol) if mol else np.nan for mol in mols],
#         'NumRotatableBonds': [Descriptors.NumRotatableBonds(mol) if mol else np.nan for mol in mols],
#         'RingCount': [mol.GetRingInfo().NumRings() if mol else np.nan for mol in mols],
#         'HeavyAtomCount': [mol.GetNumHeavyAtoms() if mol else np.nan for mol in mols],
#         'FractionCSP3': [rdMolDescriptors.CalcFractionCSP3(mol) if mol else np.nan for mol in mols],
#         'NumAliphaticRings': [rdMolDescriptors.CalcNumAliphaticRings(mol) if mol else np.nan for mol in mols],
#         'NumAromaticRings': [rdMolDescriptors.CalcNumAromaticRings(mol) if mol else np.nan for mol in mols]
#     }
#     return pd.DataFrame(features)

# # ✅ Morgan Fingerprint
# def get_morgan_fingerprint(smiles, radius=2, nBits=2048):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol:
#         fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
#         return np.array(fp)
#     else:
#         return np.zeros(nBits)

# def extract_morgan_df(df, nBits=2048):
#     fps = df['Canonical_Smiles'].apply(lambda x: get_morgan_fingerprint(x, nBits=nBits))
#     return pd.DataFrame(fps.tolist(), columns=[f'MFP_{i}' for i in range(nBits)])

# # ✅ Feature 결합
# X_train_rdkit = extract_rdkit_features(train)
# X_test_rdkit = extract_rdkit_features(test)
# X_train_morgan = extract_morgan_df(train)
# X_test_morgan = extract_morgan_df(test)

# X_train = pd.concat([X_train_rdkit, X_train_morgan], axis=1)
# X_test = pd.concat([X_test_rdkit, X_test_morgan], axis=1)
# y_train = train["Inhibition"]

# # ✅ 정규화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
# X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# # ✅ Sample Weight
# sample_weight = np.log1p(y_train)

# # ✅ 앙상블 초기화
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# oof_cat = np.zeros(len(X_train))
# oof_lgb = np.zeros(len(X_train))
# oof_xgb = np.zeros(len(X_train))
# test_cat = np.zeros(len(X_test))
# test_lgb = np.zeros(len(X_test))
# test_xgb = np.zeros(len(X_test))

# # ✅ 교차검증 모델 학습
# for train_idx, val_idx in kf.split(X_train_df):
#     X_tr, X_val = X_train_df.iloc[train_idx], X_train_df.iloc[val_idx]
#     y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
#     w_tr = sample_weight.iloc[train_idx]

#     # ✅ CatBoost (GPU)
#     cat = CatBoostRegressor(
#         iterations=1500, learning_rate=0.03, depth=6, verbose=0,
#         early_stopping_rounds=100, random_state=42, task_type='GPU'
#     )
#     cat.fit(Pool(X_tr, y_tr, weight=w_tr), eval_set=Pool(X_val, y_val))
#     oof_cat[val_idx] = cat.predict(X_val)
#     test_cat += cat.predict(X_test_df) / kf.n_splits

#     # ✅ LightGBM (GPU)
#     lgb = LGBMRegressor(
#         n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42,
#         device='gpu'
#     )
#     lgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_lgb[val_idx] = lgb.predict(X_val)
#     test_lgb += lgb.predict(X_test_df) / kf.n_splits

#     # ✅ XGBoost (GPU)
#     xgb = XGBRegressor(
#         n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42,
#         tree_method='gpu_hist', predictor='gpu_predictor'
#     )
#     xgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_xgb[val_idx] = xgb.predict(X_val)
#     test_xgb += xgb.predict(X_test_df) / kf.n_splits

# # ✅ 메타 모델 (Ridge)
# stacked_train = np.vstack([oof_cat, oof_lgb, oof_xgb]).T
# stacked_test = np.vstack([test_cat, test_lgb, test_xgb]).T
# meta_model = Ridge(alpha=1.0)
# meta_model.fit(stacked_train, y_train)
# final_preds = meta_model.predict(stacked_test)

# # ✅ 성능 평가
# rmse = np.sqrt(mean_squared_error(y_train, meta_model.predict(stacked_train)))
# mae = mean_absolute_error(y_train, meta_model.predict(stacked_train))
# print(f"✅ Meta Model RMSE: {rmse:.4f}")
# print(f"✅ Meta Model MAE : {mae:.4f}")

# # ✅ 제출
# submission['Inhibition'] = final_preds
# submission.to_csv("submission_stacking_rdkit_morgan.csv", index=False)
# print("✅ 'submission_stacking_rdkit_morgan.csv' 저장 완료!")


In [ ]:
# # ✅ 전체 실행 코드
# import os, gc, warnings
# warnings.filterwarnings("ignore")
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# from sklearn.linear_model import Ridge
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
# from xgboost import XGBRegressor
# from rdkit import Chem
# from rdkit.Chem import AllChem, Descriptors
# import optuna

# # ✅ 경로 설정
# extract_path = "/content/data_1"
# def get_path(filename): return os.path.join(extract_path, filename)
# train = pd.read_csv(get_path("train.csv"))
# test = pd.read_csv(get_path("test.csv"))
# submission = pd.read_csv(get_path("sample_submission.csv"))

# # ✅ RDKit Feature 생성
# def calc_physchem(smiles):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol is None: return [np.nan] * 10
#     return [
#         Descriptors.MolWt(mol), Descriptors.MolLogP(mol), Descriptors.NumHDonors(mol),
#         Descriptors.NumHAcceptors(mol), Descriptors.TPSA(mol), Descriptors.NumRotatableBonds(mol),
#         Descriptors.RingCount(mol), Descriptors.FractionCSP3(mol),
#         Descriptors.HeavyAtomCount(mol), Descriptors.MolMR(mol)
#     ]

# physchem_cols = ['MolWt','LogP','HDonors','HAcceptors','TPSA','RotBonds','Rings','CSP3','HeavyAtoms','MR']
# train_physchem = pd.DataFrame([calc_physchem(smi) for smi in train['Canonical_Smiles']], columns=physchem_cols)
# test_physchem = pd.DataFrame([calc_physchem(smi) for smi in test['Canonical_Smiles']], columns=physchem_cols)

# # ✅ Morgan Fingerprints
# def morgan_fp(smiles, radius=2, nBits=1024):
#     mol = Chem.MolFromSmiles(smiles)
#     return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits) if mol else [0]*nBits

# train_morgan = np.array([morgan_fp(smi) for smi in train['Canonical_Smiles']])
# test_morgan = np.array([morgan_fp(smi) for smi in test['Canonical_Smiles']])

# # ✅ Feature 결합 + 정규화
# X_train = np.hstack([train_physchem.values, train_morgan])
# X_test = np.hstack([test_physchem.values, test_morgan])
# y_train = train["Inhibition"].values
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # ✅ CatBoost Optuna 튜닝
# def objective_catboost(trial):
#     model = CatBoostRegressor(
#         iterations=trial.suggest_int("iterations", 300, 700),
#         depth=trial.suggest_int("depth", 4, 10),
#         learning_rate=trial.suggest_float("lr", 0.01, 0.2),
#         l2_leaf_reg=trial.suggest_float("l2", 1e-3, 10.0),
#         loss_function="RMSE", task_type="GPU", verbose=0
#     )
#     scores = []
#     kf = KFold(n_splits=3, shuffle=True, random_state=42)
#     for tr_idx, val_idx in kf.split(X_train):
#         model.fit(X_train[tr_idx], y_train[tr_idx])
#         pred = model.predict(X_train[val_idx])
#         scores.append(np.sqrt(mean_squared_error(y_train[val_idx], pred)))
#     return np.mean(scores)

# study = optuna.create_study(direction="minimize")
# study.optimize(objective_catboost, n_trials=20)
# best_cat_params = study.best_trial.params
# best_cat_params_fixed = {
#     'iterations': best_cat_params['iterations'],
#     'depth': best_cat_params['depth'],
#     'learning_rate': best_cat_params['lr'],
#     'l2_leaf_reg': best_cat_params['l2']
# }

# # ✅ Base 모델 학습 + OOF
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# oof_train = np.zeros((X_train.shape[0], 3))
# oof_test = np.zeros((X_test.shape[0], 3))

# for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train)):
#     print(f"📦 Fold {fold+1}")
#     cat = CatBoostRegressor(**best_cat_params_fixed, loss_function="RMSE", task_type="GPU", verbose=0)
#     cat.fit(X_train[tr_idx], y_train[tr_idx])
#     oof_train[val_idx, 0] = cat.predict(X_train[val_idx])
#     oof_test[:, 0] += cat.predict(X_test) / kf.n_splits

#     lgb = LGBMRegressor(device='gpu', learning_rate=0.05, n_estimators=500)
#     lgb.fit(X_train[tr_idx], y_train[tr_idx])
#     oof_train[val_idx, 1] = lgb.predict(X_train[val_idx])
#     oof_test[:, 1] += lgb.predict(X_test) / kf.n_splits

#     xgb = XGBRegressor(tree_method='gpu_hist', learning_rate=0.05, n_estimators=500)
#     xgb.fit(X_train[tr_idx], y_train[tr_idx])
#     oof_train[val_idx, 2] = xgb.predict(X_train[val_idx])
#     oof_test[:, 2] += xgb.predict(X_test) / kf.n_splits

# # ✅ Ridge Meta 모델 튜닝
# def objective_meta(trial):
#     alpha = trial.suggest_float("alpha", 0.01, 10.0)
#     model = Ridge(alpha=alpha)
#     scores = []
#     kf_meta = KFold(n_splits=3, shuffle=True, random_state=42)
#     for tr_meta, val_meta in kf_meta.split(oof_train):
#         model.fit(oof_train[tr_meta], y_train[tr_meta])
#         pred = model.predict(oof_train[val_meta])
#         scores.append(np.sqrt(mean_squared_error(y_train[val_meta], pred)))
#     return np.mean(scores)

# meta_study = optuna.create_study(direction="minimize")
# meta_study.optimize(objective_meta, n_trials=20)
# best_alpha = meta_study.best_params["alpha"]

# # ✅ 최종 예측 및 저장
# meta_model = Ridge(alpha=best_alpha)
# meta_model.fit(oof_train, y_train)
# final_pred = meta_model.predict(oof_test)
# submission["Inhibition"] = final_pred
# submission.to_csv("submission_stacking_optuna_ridge.csv", index=False)
# print("✅ 저장 완료: submission_stacking_optuna_ridge.csv")
# print("✅ Meta RMSE:", mean_squared_error(y_train, meta_model.predict(oof_train), squared=False))
# print("✅ Meta MAE :", mean_absolute_error(y_train, meta_model.predict(oof_train)))


In [1]:
# ✅ 1. 라이브러리 불러오기
import os, warnings, gc
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import ExtraTreesRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import optuna
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors

# ✅ 2. 경로 설정
extract_path = "/content/data_1"
def get_path(filename): return os.path.join(extract_path, filename)
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

# ✅ 3. Feature Engineering
## PhysChem
physchem_cols = ['MolWt','LogP','HDonors','HAcceptors','TPSA','RotBonds','Rings','CSP3','HeavyAtoms','MR']
def calc_physchem(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [np.nan] * len(physchem_cols)
    return [
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumHAcceptors(mol),
        Descriptors.TPSA(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.RingCount(mol),
        Descriptors.FractionCSP3(mol),
        Descriptors.HeavyAtomCount(mol),
        Descriptors.MolMR(mol)
    ]

## Morgan
def morgan_fp(smiles, radius=2, nBits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [0]*nBits
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)

train_physchem = pd.DataFrame([calc_physchem(smi) for smi in train['Canonical_Smiles']], columns=physchem_cols)
test_physchem = pd.DataFrame([calc_physchem(smi) for smi in test['Canonical_Smiles']], columns=physchem_cols)
train_morgan = np.array([morgan_fp(smi) for smi in train['Canonical_Smiles']])
test_morgan = np.array([morgan_fp(smi) for smi in test['Canonical_Smiles']])

# ✅ 4. 데이터 결합 및 정규화
X_train = np.hstack([train_physchem.values, train_morgan])
X_test = np.hstack([test_physchem.values, test_morgan])
y_train_raw = train["Inhibition"].values
y_train = np.log1p(y_train_raw)  # 로그 변환
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ✅ 5. Base Model 튜닝 (Optuna)
def tune_base_model(model_name):
    def objective(trial):
        if model_name == 'cat':
            model = CatBoostRegressor(
                iterations=trial.suggest_int("iterations", 300, 700),
                depth=trial.suggest_int("depth", 4, 10),
                learning_rate=trial.suggest_float("learning_rate", 0.01, 0.1),
                l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
                task_type="GPU", verbose=0, random_state=42
            )
        elif model_name == 'lgbm':
            model = LGBMRegressor(
                n_estimators=trial.suggest_int("n_estimators", 300, 700),
                max_depth=trial.suggest_int("max_depth", 4, 10),
                learning_rate=trial.suggest_float("learning_rate", 0.01, 0.1),
                reg_lambda=trial.suggest_float("reg_lambda", 1.0, 10.0),
                device="gpu", random_state=42
            )
        elif model_name == 'xgb':
            model = XGBRegressor(
                n_estimators=trial.suggest_int("n_estimators", 300, 700),
                max_depth=trial.suggest_int("max_depth", 4, 10),
                learning_rate=trial.suggest_float("learning_rate", 0.01, 0.1),
                reg_lambda=trial.suggest_float("reg_lambda", 1.0, 10.0),
                tree_method="gpu_hist", random_state=42
            )

        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        scores = []
        for tr_idx, val_idx in kf.split(X_train):
            model.fit(X_train[tr_idx], y_train[tr_idx])
            pred = model.predict(X_train[val_idx])
            scores.append(np.sqrt(mean_squared_error(y_train[val_idx], pred)))
        return np.mean(scores)

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)
    return study.best_params

cat_params = tune_base_model("cat")
lgbm_params = tune_base_model("lgbm")
xgb_params = tune_base_model("xgb")

# ✅ 6. Base 모델 학습 & OOF 예측
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_train = np.zeros((X_train.shape[0], 4))
oof_test = np.zeros((X_test.shape[0], 4))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"📦 Fold {fold+1}")

    cat = CatBoostRegressor(**cat_params, loss_function="RMSE", task_type="GPU", verbose=0, random_state=42)
    cat.fit(X_train[tr_idx], y_train[tr_idx])
    oof_train[val_idx, 0] = cat.predict(X_train[val_idx])
    oof_test[:, 0] += cat.predict(X_test) / kf.n_splits

    lgbm = LGBMRegressor(**lgbm_params, device='gpu', random_state=42)
    lgbm.fit(X_train[tr_idx], y_train[tr_idx])
    oof_train[val_idx, 1] = lgbm.predict(X_train[val_idx])
    oof_test[:, 1] += lgbm.predict(X_test) / kf.n_splits

    xgb = XGBRegressor(**xgb_params, tree_method='gpu_hist', random_state=42)
    xgb.fit(X_train[tr_idx], y_train[tr_idx])
    oof_train[val_idx, 2] = xgb.predict(X_train[val_idx])
    oof_test[:, 2] += xgb.predict(X_test) / kf.n_splits

    ext = ExtraTreesRegressor(n_estimators=300, random_state=42)
    ext.fit(X_train[tr_idx], y_train[tr_idx])
    oof_train[val_idx, 3] = ext.predict(X_train[val_idx])
    oof_test[:, 3] += ext.predict(X_test) / kf.n_splits

# ✅ 7. Meta 모델 튜닝 (Optuna Ridge)
def objective_meta(trial):
    alpha = trial.suggest_float("alpha", 0.01, 10.0)
    model = Ridge(alpha=alpha, random_state=42)
    kf_meta = KFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for tr_meta, val_meta in kf_meta.split(oof_train):
        model.fit(oof_train[tr_meta], y_train[tr_meta])
        pred = model.predict(oof_train[val_meta])
        scores.append(np.sqrt(mean_squared_error(y_train[val_meta], pred)))
    return np.mean(scores)

meta_study = optuna.create_study(direction="minimize")
meta_study.optimize(objective_meta, n_trials=20)
best_alpha = meta_study.best_params["alpha"]

# ✅ 8. Meta 모델 훈련 & 예측
meta_model = Ridge(alpha=best_alpha, random_state=42)
meta_model.fit(oof_train, y_train)
final_pred = np.expm1(meta_model.predict(oof_test))  # 역변환

# ✅ 9. 저장
submission["Inhibition"] = final_pred
submission.to_csv("submission_stack_optuna_final.csv", index=False)
print("✅ 저장 완료: submission_stack_optuna_final.csv")


[I 2025-07-10 07:38:16,809] A new study created in memory with name: no-name-71c7cac9-efc7-46f6-a27a-e826ea6da4b1
[I 2025-07-10 07:38:23,684] Trial 0 finished with value: 1.2580830058569694 and parameters: {'iterations': 489, 'depth': 6, 'learning_rate': 0.08960608652270292, 'l2_leaf_reg': 3.3372732508530207}. Best is trial 0 with value: 1.2580830058569694.
[I 2025-07-10 07:38:30,113] Trial 1 finished with value: 1.2482973904181602 and parameters: {'iterations': 450, 'depth': 6, 'learning_rate': 0.08392683516219326, 'l2_leaf_reg': 3.189391000375669}. Best is trial 1 with value: 1.2482973904181602.
[I 2025-07-10 07:38:35,939] Trial 2 finished with value: 1.2472073780106476 and parameters: {'iterations': 464, 'depth': 4, 'learning_rate': 0.08181490818212969, 'l2_leaf_reg': 6.4829734589622765}. Best is trial 2 with value: 1.2472073780106476.
[I 2025-07-10 07:38:41,399] Trial 3 finished with value: 1.237378939151424 and parameters: {'iterations': 374, 'depth': 4, 'learning_rate': 0.0749872

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3133
[LightGBM] [Info] Number of data points in the train set: 1120, number of used features: 647
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-40GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 26 dense feature groups (0.03 MB) transferred to GPU in 0.001749 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 3.003146
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

[I 2025-07-10 07:47:03,905] Trial 0 finished with value: 1.2676991081384408 and parameters: {'n_estimators': 368, 'max_depth': 4, 'learning_rate': 0.07669803152742115, 'reg_lambda': 9.82538404735388}. Best is trial 0 with value: 1.2676991081384408.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-07-10 07:47:06,135] Trial 1 finished with value: 1.3100718985488873 and parameters: {'n_estimators': 491, 'max_depth': 7, 'learning_rate': 0.08237103602959785, 'reg_lambda': 5.803155199700003}. Best is trial 0 with value: 1.2676991081384408.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-07-10 07:47:50,790] Trial 19 finished with value: 1.2544398066663536 and parameters: {'n_estimators': 435, 'max_depth': 6, 'learning_rate': 0.01946828668560114, 'reg_lambda': 7.016140169420555}. Best is trial 15 with value: 1.2448722908606458.
[I 2025-07-10 07:47:50,791] A new study created in memory with name: no-name-f3b8e487-d9c7-4e8f-8cd7-a9a62776e655
[I 2025-07-10 07:47:54,097] Trial 0 finished with value: 1.2700523658625091 and parameters: {'n_estimators': 594, 'max_depth': 5, 'learning_rate': 0.015612988533547954, 'reg_lambda': 1.3205425373661068}. Best is trial 0 with value: 1.2700523658625091.
[I 2025-07-10 07:47:58,863] Trial 1 finished with value: 1.285266340275995 and parameters: {'n_estimators': 461, 'max_depth': 10, 'learning_rate': 0.08733564417677935, 'reg_lambda': 4.807584986544199}. Best is trial 0 with value: 1.2700523658625091.
[I 2025-07-10 07:48:05,299] Trial 2 finished with value: 1.2817739774768862 and parameters: {'n_estimators': 693, 'max_depth': 9, 'l

📦 Fold 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3509
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 770
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-40GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 25 dense feature groups (0.04 MB) transferred to GPU in 0.001419 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 3.005373
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

[I 2025-07-10 07:53:40,012] A new study created in memory with name: no-name-8ee06de5-fcff-4914-bfb0-fb13d70b46f7
[I 2025-07-10 07:53:40,035] Trial 0 finished with value: 1.2310529105312835 and parameters: {'alpha': 3.9439354938196667}. Best is trial 0 with value: 1.2310529105312835.
[I 2025-07-10 07:53:40,042] Trial 1 finished with value: 1.2310796142006968 and parameters: {'alpha': 8.366768713721592}. Best is trial 0 with value: 1.2310529105312835.
[I 2025-07-10 07:53:40,048] Trial 2 finished with value: 1.2311003671937406 and parameters: {'alpha': 1.508004967631543}. Best is trial 0 with value: 1.2310529105312835.
[I 2025-07-10 07:53:40,054] Trial 3 finished with value: 1.2311023559894265 and parameters: {'alpha': 9.565010040487952}. Best is trial 0 with value: 1.2310529105312835.
[I 2025-07-10 07:53:40,062] Trial 4 finished with value: 1.2310707084068315 and parameters: {'alpha': 2.6451000163292377}. Best is trial 0 with value: 1.2310529105312835.
[I 2025-07-10 07:53:40,071] Trial 

✅ 저장 완료: submission_stack_optuna_final.csv
